# PC2 - Grupo 2 (Python)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

#### 1. Loading and processing the data


In [2]:
import numpy as np
import pandas as pd

In [3]:
data= "https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"

df =pd.read_csv(data)

In [4]:
df = df.set_index('rownames')

#### As in Group Assignment 1 2024 - 2 #1044 , generate the extra-flexible model. This means that it contains all two-way interactions between the experience polynomials and the indicator variables

In [7]:
basic_predictors = ["sex", "hsg", "scl", "clg", "ad", "so", "we", "ne", "exp1", "occ2", "ind2"]
df_with_dummies = pd.get_dummies(df[basic_predictors], columns = ["occ2", "ind2"], drop_first = True, dtype = int) 
X_extra_flexible = df_with_dummies.copy()
X_extra_flexible = pd.concat((X_extra_flexible, df[["exp2", "exp3", "exp4"]]), axis = 1)
extra_flexible_base_features = X_extra_flexible.columns[1:]


In [8]:
base_experience_variables = ["exp1", "exp2", "exp3"]
exp_4 = ["exp4"]
education_variables = ["hsg", "scl", "clg", "ad"]
location_variables = ["so", "we", "ne"]
occupation_variables = [variable for variable in X_extra_flexible.columns if "occ2" in variable]
industry_variables = [variable for variable in X_extra_flexible.columns if "ind2" in variable]
variable_groups = [base_experience_variables, exp_4, education_variables, location_variables, occupation_variables, industry_variables]

In [9]:
for variable_group_index, variable_group in enumerate(variable_groups, 1):
        if variable_group == len(variable_groups):
                break

        other_variable_groups = variable_groups[variable_group_index:]

        for variable in variable_group:
                for other_variables in other_variable_groups:
                        temp = X_extra_flexible[other_variables].multiply(X_extra_flexible[variable], axis = "index")
                        temp = temp.add_prefix(f"{variable}")
                        X_extra_flexible = pd.concat(
                                (X_extra_flexible, temp),
                                axis = 1
                        )

In [10]:
temp = pd.DataFrame({"exp8": X_extra_flexible["exp4"] ** 2})
extra_flexible_model_array = pd.concat((X_extra_flexible, temp), axis = 1)

In [13]:
print(extra_flexible_model_array.shape)

(5150, 968)


#### 2.1. Generate the array for the outcome variable Y and normalize it

In [11]:
df_logwage = df[['lwage']]
log_w = np.array(df_logwage['lwage'])
log_w = log_w.reshape(-1, 1)

In [12]:
norm_log_w = (log_w - np.mean(log_w)) / np.std(log_w)

print(norm_log_w)

[[-1.24037498]
 [ 1.5815695 ]
 [-0.99532021]
 ...
 [ 1.19031569]
 [ 0.9200321 ]
 [-0.20976589]]


In [14]:
continuous_columns = base_experience_variables + exp_4

In [17]:
for var in continuous_columns:
    if var in extra_flexible_model_array.columns:
        mean = extra_flexible_model_array[var].mean()
        std = extra_flexible_model_array[var].std()
        extra_flexible_model_array[var] = (extra_flexible_model_array[var] - mean) / std

# Verifica que se aplicó la normalización correctamente
print(extra_flexible_model_array[continuous_columns])

              exp1      exp2      exp3      exp4
rownames                                        
10       -0.637222 -0.632088 -0.544750 -0.464746
12        1.624909  1.647396  1.487693  1.256002
15        0.399588  0.055256 -0.165910 -0.273125
18        1.059376  0.807586  0.509984  0.260497
19        0.776610  0.455166  0.166481 -0.031616
...            ...       ...       ...       ...
32620    -0.448711 -0.552106 -0.518109 -0.456974
32624    -0.165945 -0.394642 -0.449160 -0.430494
32626    -0.260200 -0.452129 -0.476561 -0.441880
32631    -0.354455 -0.504617 -0.499405 -0.450550
32643     0.022566 -0.264671 -0.379038 -0.397466

[5150 rows x 4 columns]


#### Split between training and testing samples. The testing sample should be 10% of the total.

In [18]:
from sklearn.model_selection import train_test_split
ef_model =extra_flexible_model_array
y=norm_log_w

In [19]:
ef_train,ef_test,y_ef_train,y_ef_test = train_test_split(ef_model,y, train_size= 0.1)


### 3. Creating the Lasso Cross - Validation Procedure 

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for λ values to try during cross-validation.

In [20]:
import numpy as np

def log_spaced_grid(lower_bound, upper_bound, log_spacing, num_points):
    log_lower = np.log(lower_bound)
    log_upper = np.log(upper_bound)

 
    log_grid = np.linspace(log_lower, log_upper, num_points)
    
   
    return np.exp(log_grid)


5. Program a function to generate 
k
 folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of 
k
 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [21]:
def generate_k_folds(X, k):
    n_samples = X.shape[0]
    fold_sizes = np.full(k, n_samples // k, dtype=int)
    fold_sizes[:n_samples % k] += 1  # Distribute remaining samples
    
    # Create an array of indices and shuffle it
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Create the folds
    folds = []
    current = 0
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        mask = np.zeros(n_samples, dtype=bool)
        mask[indices[start:stop]] = True
        folds.append(mask)
        current = stop
    
    return folds

6. Program a function that integrates those that you programmed in the last two items to find the value of 
λ
 that minimizes the testing mean square error across folds.

In [22]:
from sklearn.linear_model import Lasso

def find_optimal_lambda(Y, X, lambda_bounds, k):
    lambdas = log_spaced_grid(lambda_bounds[0], lambda_bounds[1])
    
    folds = generate_k_folds(X, k)
    
    all_mse = np.zeros((len(lambdas), k))
    
    for i, lambda_val in enumerate(lambdas):
        for j, fold in enumerate(folds):
            X_train, X_test = X[~fold], X[fold]
            Y_train, Y_test = Y[~fold], Y[fold]
            
            model = Lasso(alpha=lambda_val, fit_intercept=True)
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            all_mse[i, j] = np.mean((Y_test - Y_pred)**2)

    avg_mse = np.mean(all_mse, axis=1)
    
    
    optimal_index = np.argmin(avg_mse)
    optimal_lambda = lambdas[optimal_index]
    
    optimal_model = Lasso(alpha=optimal_lambda, fit_intercept=True)
    optimal_model.fit(X, Y)
    
    
    result = {
        'optimal_lambda': optimal_lambda,
        'optimal_coef': optimal_model.coef_,
        'all_lambdas': lambdas,
        'all_mse': avg_mse
    }
    
    return result

7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

In [23]:

def lasso_predict(optimal_model, X):
    optimal_lambda = optimal_model['optimal_lambda']
    optimal_coef = optimal_model['optimal_coef']

    model = Lasso(alpha=optimal_lambda, fit_intercept=True)
    model.coef_ = optimal_coef
    model.intercept_ = 0  

    predictions = model.predict(X)
    
    return predictions

# 3.Applying the Lasso Cross-Validation Procedure

8. Fit a simple OLS model with the training sample.

In [25]:
import statsmodels.api as sm

X_train = sm.add_constant(ef_train)

# Create and fit the OLS model
model = sm.OLS(y_ef_train, X_train)
results = model.fit()

# Print the summary of the model
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     1.474
Date:                Sat, 21 Sep 2024   Prob (F-statistic):             0.0396
Time:                        20:50:24   Log-Likelihood:                -55.497
No. Observations:                 515   AIC:                             1033.
Df Residuals:                      54   BIC:                             2990.
Df Model:                         460                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             28.4014     94.518      0.

9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. The natural logarithm of the numbers in the grid of $\lambda$s should be spaced by 0.1, and the natural logarithm of the lower and upper bounds of the grid should be -7 and 7 respectively. For the k-folds procedure, use 5 folds. Print the lambda and the coefficients.

In [27]:
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split


# Definir los límites de la grilla de lambda en el espacio logarítmico
lambda_bounds = (np.exp(-7.0), np.exp(7.0))

# Crear una grilla de valores de lambda (en scikit-learn esto se llama alpha)
lambda_grid = np.exp(np.arange(-7, 7, 0.1))

# Ajustar el modelo Lasso con validación cruzada de 5 folds
lasso_cv = LassoCV(alphas=lambda_grid, cv=5, random_state=42).fit(ef_train, y_ef_train)

# Obtener el mejor valor de lambda (alpha)
optimal_lambda = lasso_cv.alpha_
print(f"Optimal lambda (alpha): {optimal_lambda}")

# Obtener los coeficientes asociados con el mejor lambda
optimal_coefficients = lasso_cv.coef_
print(f"Coefficients: {optimal_coefficients}")

c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1656: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.305e+00, tolerance: 3.947e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Nicolas Silva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regu

Optimal lambda (alpha): 0.110803158362332
Coefficients: [-0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.99756792e-04 -3.15438367e-02 -9.30264016e-03
  2.04042719e-02  3.73792905e-02 -